In [ ]:
from pytket import Circuit, OpType
from pytket.extensions.qiskit import AerBackend
circ = Circuit(3)
circ.CZ(0, 1)
circ.H(1)
circ.Rx(0.42, 1)
circ.S(1)
circ.add_gate(OpType.YYPhase, 0.96, [1, 2])
circ.CX(0, 1)
circ.measure_all()
#b = AerBackend()
b= IBMQBackend("ibmq_belem")
for ol in range(3):
    test = circ.copy()
    b.default_compilation_pass(ol).apply(test)
    assert b.valid_circuit(test)
    print("Optimisation level", ol)
    print("Gates", test.n_gates)
    print("CXs", test.n_gates_of_type(OpType.CX))


In [ ]:
c1 = Circuit(3)
c1.depth() 



In [ ]:
c1.CX(0,1)
c1.CX(1,2)
c1.CX(2,0)
c1.depth()

In [ ]:
render_circuit_jupyter(c1)

In [ ]:
measure_all(c1)

In [ ]:
c2 = Circuit(3)

c2.CX(0,1)
c2.Z(1)
c2.CX(1,2)

In [ ]:
render_circuit_jupyter(c2)

In [ ]:
c1.depth_by_type(OpType.CX)

### Embedding into Qiskit




Not only is the goal of tket to be a device-agnostic platform, but also interface-agnostic, so users are not obliged to have to work entirely in tket to benefit from the wide range of devices supported. For example, Qiskit is currently the most widely adopted quantum software development platform, providing its own modules for building and compiling circuits, submitting to backends, applying error mitigation techniques and combining these into higher-level algorithms. Each Backend in pytket can be wrapped up to imitate a Qiskit backend, allowing the benefits of tket to be felt in existing Qiskit projects with minimal work.

In [ ]:
from qiskit.utils import QuantumInstance
from qiskit.algorithms import Grover, AmplificationProblem
from qiskit.circuit import QuantumCircuit

from pytket.extensions.qulacs import QulacsBackend
from pytket.extensions.qiskit.tket_backend import TketBackend

b = QulacsBackend()
backend = TketBackend(b, b.default_compilation_pass())
qinstance = QuantumInstance(backend)

oracle = QuantumCircuit(2)
oracle.cz(0, 1)

def is_good_state(bitstr):
    return sum(map(int, bitstr)) == 2

problem = AmplificationProblem(oracle=oracle, is_good_state=is_good_state)
grover = Grover(quantum_instance=qinstance)
result = grover.amplify(problem)
print("Top measurement:", result.top_measurement)

In [ ]:
oracle.draw(output='mpl')


In [ ]:
from pytket.qasm import circuit_from_qasm, circuit_to_qasm_str
import tempfile, os

fd, path = tempfile.mkstemp(".qasm")
os.write(fd, """OPENQASM 2.0;
include "qelib1.inc";
qreg q[2];
creg c[2];
h q[0];
cx q[0], q[1];
cz q[1], q[0];
measure q -> c;
""".encode())
os.close(fd)
circ = circuit_from_qasm(path)
os.remove(path)

print(circuit_to_qasm_str(circ))


In [ ]:
render_circuit_jupyter(circ)

In [ ]:
from pytket import Circuit, OpType
from sympy import Symbol
a = Symbol("alpha")
b = Symbol("beta")
circ = Circuit(2)
circ.Rx(a, 0)
circ.Rx(-2*a, 1)
circ.CX(0, 1)
circ.add_gate(OpType.YYPhase, b, [0, 1])
print(circ.get_commands())

s_map = {a: 0.3, b:1.25}
circ.symbol_substitution(s_map)
print(circ.get_commands())

In [ ]:
render_circuit_jupyter(circ)

In [ ]:
tk_to_qiskit(circ).draw(output='mpl')

In [ ]:
from pytket.extensions.qiskit import IBMQBackend, AerStateBackend
dev_b = IBMQBackend("ibmq_athens")
sim_b = AerStateBackend()
print(dev_b.required_predicates)
print(sim_b.required_predicates)

### Compilation

So far, we have already covered enough to be able to design the Circuit s we want to run, submit them to a Backend, and interpret the results in a meaningful way. This is all you need if you want to just try out a quantum computer, run some toy examples and observe some basic results. We actually glossed over a key step in this process by using the Backend.compile_circuit() method. The compilation step maps from the universal computer abstraction presented at Circuit construction to the restricted fragment supported by the target Backend, and knowing what a compiler can do to your program can help reduce the burden of design and improve performance on real devices.

The necessity of compilation maps over from the world of classical computation: it is much easier to design correct programs when working with higher-level constructions that aren’t natively supported, and it shouldn’t require a programmer to be an expert in the exact device architecture to achieve good performance. There are many possible low-level implementations on the device for each high-level program, which vary in the time and resources taken to execute. However, because QPUs are analog devices, the implementation can have a massive impact on the quality of the final outcomes as a result of changing how susceptible the system is to noise. Using a good compiler and choosing the methods appropriately can automatically find a better low-level implementation. Each aspect of the compilation procedure is exposed through pytket to provide users with a way to have full control over what is applied and how.

The primary goals of compilation are two-fold: solving the constraints of the Backend to get from the abstract model to something runnable, and optimising/simplifying the Circuit to make it faster, smaller, and less prone to noise. Every step in compilation can generally be split up into one of these two categories (though even the constraint solving steps could have multiple solutions over which we could optimise for noise).

Each compiler pass inherits from the BasePass class, capturing a method of transforming a Circuit. The main functionality is built into the BasePass.apply() method, which applies the transformation to a Circuit in-place. The Backend.compile_circuit() method is simply an alias for BasePass.apply() from the Backend ‘s recommended pass sequence. This chapter will explore these compiler passes, the different kinds of constraints they are used to solve and optimisations they apply, to help you identify which ones are appropriate for a given task.



### Rebases
One of the simplest constraints to solve for is the GateSetPredicate, since we can just substitute each gate in a Circuit with an equivalent sequence of gates in the target gateset according to some known gate decompositions. In pytket, such passes are referred to as “rebases”. The intention here is to perform this translation naively, leaving the optimisation of gate sequences to other passes. Rebases can be applied to any Circuit and will preserve every structural Predicate, only changing the types of gates used.



In [ ]:
from pytket import Circuit
from pytket.passes import RebaseIBM
circ = Circuit(2, 2)
circ.Rx(0.3, 0).Ry(-0.9, 1).CZ(0, 1).S(0).CX(1, 0).measure_all()

RebaseIBM().apply(circ)

print(circ.get_commands())


In [ ]:
from pytket import Circuit, OpType
from pytket.predicates import GateSetPredicate, NoMidMeasurePredicate
circ = Circuit(2, 2)
circ.Rx(0.2, 0).CX(0, 1).Rz(-0.7, 1).measure_all()

gateset = GateSetPredicate({OpType.Rx, OpType.CX, OpType.Rz, OpType.Measure})
midmeasure = NoMidMeasurePredicate()

print(gateset.verify(circ))
print(midmeasure.verify(circ))



In [ ]:
tk_to_qiskit(circ).draw(output='mpl')

In [ ]:
circ.S(0)

print(gateset.verify(circ))
print(midmeasure.verify(circ))


In [ ]:
tk_to_qiskit(circ).draw(output='mpl')

frqi - qc1 circuit - max intensity for all the pixels

In [ ]:
import qiskit as qk
from qiskit import QuantumCircuit, Aer, IBMQ
from qiskit import transpile, assemble
from qiskit.tools.jupyter import *
from qiskit.visualization import plot_histogram
from math import pi

In [ ]:
theta = pi/2  # all pixels white
qc1 = QuantumCircuit(3)

qc1.h(0)
qc1.h(1)

qc1.barrier()
#Pixel 1

qc1.cry(theta,0,2)
qc1.cx(0,1)
qc1.cry(-theta,1,2)
qc1.cx(0,1)
qc1.cry(theta,1,2)

qc1.barrier()
#Pixel 2

qc1.x(1)

qc1.cry(theta,0,2)
qc1.cx(0,1)
qc1.cry(-theta,1,2)
qc1.cx(0,1)
qc1.cry(theta,1,2)

qc1.barrier()

qc1.x(1)
qc1.x(0)
qc1.cry(theta,0,2)
qc1.cx(0,1)
qc1.cry(-theta,1,2)
qc1.cx(0,1)
qc1.cry(theta,1,2)


qc1.barrier()

qc1.x(1)

qc1.cry(theta,0,2)
qc1.cx(0,1)
qc1.cry(-theta,1,2)
qc1.cx(0,1)
qc1.cry(theta,1,2)

qc1.measure_all()

qc1.draw(output='mpl')

In [ ]:
print("Depth : ", qc1.depth())
print("Operations: ", qc1.count_ops())


In [ ]:
from qiskit.compiler import transpile
from qiskit.transpiler import PassManager
from qiskit.transpiler.passes import Unroller
pass_ = Unroller(['u3', 'cx'])
pm = PassManager(pass_)
new_circ = pm.run(qc1)
#new_circ.draw()
print("Depth : ", new_circ.depth())
print("Operations: ", new_circ.count_ops())

In [ ]:
new_circ.draw(output="mpl")

comparing the unrolling from qiskit with  rebaseibm from tket 

In [ ]:
qc1tk=qiskit_to_tk(qc1)
qc1tk_back=qc1tk.copy()

In [ ]:
RebaseIBM().apply(qc1tk)
print("Depth", qc1tk.depth())
print("Gates", qc1tk.n_gates)
print("CXs", qc1tk.n_gates_of_type(OpType.CX))
print("U3ss", qc1tk.n_gates_of_type(OpType.CX))
print("Barrier", qc1tk.n_gates_of_type(OpType.Barrier))
print("Measure", qc1tk.n_gates_of_type(OpType.Measure))

In [ ]:
from qiskit.test.mock import FakeAthens
fake_athens = FakeAthens()
# The device coupling map is needed for transpiling to correct
# CNOT gates before simulation
coupling_map = fake_athens.configuration().coupling_map
optimized_3 = transpile(qc1,backend=fake_athens, seed_transpiler=11, optimization_level=3)
print('gates = ', optimized_3.count_ops())
print('depth = ', optimized_3.depth())
print('total number of gates = ', optimized_3.size())
#basis_gates=['u3', 'cx']

In [ ]:
optimized_3.draw(output='mpl')

In [ ]:
from pytket.utils import prepare_circuit

circ1=qc1tk_back
#b = AerBackend()
b = IBMQBackend("ibmq_athens")
test = circ1.copy()
#RebaseIBM().apply(test)
#b.default_compilation_pass(2).apply(test)
c0, ppcirc = prepare_circuit(test)


In [ ]:
tk_to_qiskit(c0).draw(output='mpl')

In [ ]:
b.compile_circuit(c0,2)
#b.compile_circuit(test,2)
#assert b.valid_circuit(test)
print("Optimisation level", 2)
print("Depth", c0.depth())
print("Gates", c0.n_gates)
print("CXs", c0.n_gates_of_type(OpType.CX))
print("Rzs", c0.n_gates_of_type(OpType.Rz))
print("Sxs", c0.n_gates_of_type(OpType.SX))
#print(test.get_commands())

In [ ]:
tk_to_qiskit(c0).draw(output='mpl')

In [ ]:
tk_to_qiskit(test).draw(output='mpl')

In [ ]:
tk_to_qiskit(test).draw(output='mpl')

In [ ]:
qasm_sim = Aer.get_backend('qasm_simulator')
t_qc1 = transpile(qc1, qasm_sim)
qobj = assemble(t_qc1, shots=4096)
result = qasm_sim.run(qobj).result()
counts = result.get_counts(qc1)
print(counts)
plot_histogram(counts)


showing tket added value on top of qiskit: https://github.com/CQCL/pytket/blob/master/examples/qiskit_integration.ipynb

In [ ]:
from qiskit.opflow.primitive_ops import PauliSumOp

In [ ]:
H2_op = PauliSumOp.from_list(
    [
        ("II", -1.052373245772859),
        ("IZ", 0.39793742484318045),
        ("ZI", -0.39793742484318045),
        ("ZZ", -0.01128010425623538),
        ("XX", 0.18093119978423156),
    ]
)


In [ ]:
from qiskit.algorithms import NumPyEigensolver


In [ ]:
es = NumPyEigensolver(k=1)
exact_result = es.compute_eigenvalues(H2_op).eigenvalues[0].real
print("Exact result:", exact_result)


In [ ]:
from qiskit.algorithms import VQE
from qiskit.algorithms.optimizers import SPSA
from qiskit.circuit.library import EfficientSU2

In [ ]:
def vqe_solve(op, maxiter, quantum_instance):
    optimizer = SPSA(maxiter=maxiter)
    ansatz = EfficientSU2(op.num_qubits, entanglement="linear")
    vqe = VQE(ansatz=ansatz, optimizer=optimizer, quantum_instance=quantum_instance)
    return vqe.compute_minimum_eigenvalue(op).eigenvalue


In [ ]:
from pytket.extensions.qiskit import IBMQEmulatorBackend
from qiskit import IBMQ


In [ ]:
IBMQ.load_account()
b_emu = IBMQEmulatorBackend("ibmq_belem", hub="ibm-q", group="open", project="main")


In [ ]:
from pytket.extensions.qiskit.tket_backend import TketBackend
from qiskit.utils import QuantumInstance


In [ ]:
qis_backend = TketBackend(b_emu)
qi = QuantumInstance(qis_backend, shots=8192, wait=0.1)


In [ ]:
print("VQE result:", vqe_solve(H2_op, 50, qi))


In [ ]:
from pytket.passes import FullPeepholeOptimise
from qiskit.transpiler import PassManager
from qiskit.transpiler.passes import Unroller

In [ ]:
qis_backend2 = TketBackend(b_emu, FullPeepholeOptimise())
qi2 = QuantumInstance(
    qis_backend2,
    pass_manager=PassManager(Unroller(["cx", "h", "rx", "ry", "rz"])),
    shots=8192,
    wait=0.1,
)

In [ ]:
print("VQE result (with optimisation):", vqe_solve(H2_op, 50, qi2))


run on braket

In [ ]:
from pytket import Circuit
bell_circ = Circuit(2).H(0).CX(0,1)


In [ ]:
from  pytket.extensions.braket import BraketBackend
S3_BUCKET = "amazon-braket-test"
S3_FOLDER = "test-folder"
ionq_backend = BraketBackend(
    s3_bucket=S3_BUCKET,
    s3_folder=S3_FOLDER,
    device_type="qpu",
    provider="ionq",
    device="ionQdevice",
)


In [ ]:
ionq_backend.compile_circuit(bell_circ)

In [ ]:
job_handle = ionq_backend.process_circuit(bell_circ, n_shots=20)

In [ ]:
print(ionq_backend.circuit_status(job_handle))

In [ ]:
job_handle

In [ ]:
result = ionq_backend.get_result(job_handle)


In [ ]:
result

In [ ]:
from pytket.circuit import Bit
def get_cbits(backend, circuit):
    return [Bit(backend.device().nodes.index(q)) for q in circuit.qubits]
cbits = get_cbits(ionq_backend, bell_circ)


In [ ]:
counts = result.get_counts(cbits=cbits)
print(counts)


Questions:
- how to get the task id of the job
- how to display the counts nicely (like in the notebook examples for braket) - result converter
- 

run a real circuit on the ionq device

In [ ]:
from pytket.extensions.qiskit import IBMQBackend

backend = IBMQBackend("ibmq_santiago")
for key in backend.characterisation:
    print(key)


In [ ]:
print(repr(backend.device))


In [ ]:
from pytket.routing import NoiseAwarePlacement, GraphPlacement

noise_placer = NoiseAwarePlacement(backend.device)
graph_placer = GraphPlacement(backend.device)

circ = Circuit(3).CX(0,1).CX(0,2)

print(backend.device.coupling, '\n')

noise_placement = noise_placer.get_placement_map(circ)
graph_placement = graph_placer.get_placement_map(circ)

print('NoiseAwarePlacement mapping:')
for k, v in noise_placement.items():
    print(k, v)

print('\nGraphPlacement mapping:')
for k, v in graph_placement.items():
    print(k, v)


removed stuff

<figure>
<img src="files/athens.png" width="200" height="100"
     alt="athens" >
<figcaption>IBM Athens device</figcaption>
</figure>

In [ ]:
backend = IBMQBackend("ibmq_athens")
place = PlacementPass(GraphPlacement(backend.device))
place.apply(circ)

print(circ.get_commands())
print(ConnectivityPredicate(backend.device).verify(circ))

In [ ]:
render_circuit_jupyter(circ)

A set of passes can be applied in order to fully optimize the circuit. This can be done manually byt pytket also has a default predefined sequences (i.e. levels of optimization.)
Level 0: Just solves the constraints as simply as possible. No optimisation.
Level 1: Adds basic optimisations for efficient compilation.
Level 2: Extends to more intensive optimisations.

END -----------

We now have a circuit that can actually be executed on a specific device.

As usual, if you prefer to visualize the circuit using Qiskit, we are just one line of code away